# Chapter 9 Cython Profiling Tools

### Cython Runtime Profiling

推荐使用profiling tools来量化程序的表现，以决定哪些部分需要使用Cython改写。

Python有许多方便的profiling工具：
    - 内建profile/cProfile模块。
    - Ipython构建在此之上的魔法命令%timeit(对small statement)及%run(对整个程序)
    
然而，这些profiling工具用在调用C代码的Python代码上时，所有关于C级别操作的性能信息都丢失了。为了解决这一问题，Cython可以“欺骗”这些profiling工具，让它们以为这些C级别的调用时常规的Python调用。

##### cProfiler.run()结果含义：
- ncalls：调用的次数
- tottime：函数花费的总时间（**不包括**花在called function上的时间），通常这个时间提供了最有用的信息。
- percall：第一个percall是tottime/ncalls
- cumtime：函数花费的总时间（**包括**花在called function上的时间），通常这个时间提供了最有用的信息。
- percall：第二个percall是cumtime/ncalls
- filename：模块名+行数+该行对应的函数名

##### profile编译指令
使Cython支持在生成的代码中进行runtime profiling。
- 在*.pyx*文件开头添加<font color='#ff2222' face='consolas'># cython: profile=True</font>

##### 选择要profile的函数
移除module-wide profiling指令，cimport cython魔法模块，在要profile的函数前添加<font color='#ff2222' face='consolas'>@cython.profile(True)</font>装饰器。
Cython不能对imported function进行profile，智能对extension module中定义的函数进行profile。

In [12]:
%%file integ.pyx
# cython: profile=True
# from math import sin
from libc.math cimport sin
cimport cython
cpdef double sin2(double x):
    return sin(x)**2

@cython.cdivision(True)
cpdef double integrate(double a, double b, f, int N=2000):
    cdef:
        int i
        double dx = (b-a)/N
        double s = 0.0
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

Overwriting integ.pyx


In [13]:
%%file main.py
import pyximport
pyximport.install()
from integ import sin2,integrate
from math import pi, sin
def main():
    a, b = 0.0, 2.0 * pi
    return integrate(a, b, sin2, N=400000)
if __name__ == '__main__':
    import cProfile
    cProfile.run('main()',sort='time')

Overwriting main.py


In [14]:
!python main.py
# 为什么使用integrate文件名会报错？？？

integ.c
integ.obj : warning LNK4197: export 'initinteg' specified multiple times; using first specification
   Creating library C:\Users\Think X260\.pyxbld\temp.win-amd64-2.7\Release\Users\Think X260\.pyxbld\temp.win-amd64-2.7\Release\pyrex\integ.lib and object C:\Users\Think X260\.pyxbld\temp.win-amd64-2.7\Release\Users\Think X260\.pyxbld\temp.win-amd64-2.7\Release\pyrex\integ.exp
         400005 function calls in 0.089 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   400000    0.047    0.000    0.047    0.000 integ.pyx:5(sin2)
        1    0.042    0.042    0.127    0.127 integ.pyx:9(integrate)
        1    0.000    0.000    0.127    0.127 main.py:5(main)
        1    0.000    0.000    0.127    0.127 {integ.integrate}
        1    0.000    0.000    0.127    0.127 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Performance Profiling and Annotations
使用<font face='consolas'>cProfile</font>和Cython的<font face='consolas'>profile</font>是我们应该使用的第一个profiling工具。其直接告诉我们，基于运行时表现，应该关注那些代码。

Cython还提供了*compile-time annotations*，来告诉我们为什么给定的代码速度缓慢。其和前面介绍的runtime profiling让门可以全面互补的评估Cython代码的表现。

<font face='consolas'>cython</font>编译器有一个可选的<font face='consolas'>--annotate(简写形式：-a)</font>，可以指示<font face='consolas'>cython</font>生成Cython源码的HTML文件，即*code annotation*。Cython会基于对Python/C API的调用次数对每一行代码做彩色标注：调用次数很多的代码会被高亮为<font color='#ff9911'>黄色</font>，不调用C API的代码无高亮。点击annotation file中的某行代码会展开该行生成的对应的C代码，使我们可以很方便的进行检查。

<font face='consolas'>cython</font>这样做是基于一个启发式的结论：如果一行Cython代码调用了很多次Python/C API，那么可能该行操作了许多Python对象，很多时候性能不佳。而如果一行代码只被翻译成很少的几行C代码，且未调用C API，那么其没有操作Python对象，很有可能具有较好的性能。

In [15]:
!cython -a integ.pyx